# タスクシミュレータ

In [205]:
import pandas as pd
from datetime import datetime, date, timedelta
from dateutil.relativedelta import relativedelta
import math

In [206]:
encoding = 'utf-8'

## 計画作成

### プロジェクト期間

In [207]:
StartDate = datetime(2021,9 ,15)
EndDate   = datetime(2022,1,15)

### 担当者別係数

In [208]:
# 担当者名,係数でCSV作成（ヘッダ不要）
owner_coefficient = pd.read_csv('./data/owner_coefficient.csv', header=None, index_col=0, squeeze=True, encoding=encoding).to_dict()
owner_coefficient

{'田中': 2.0, '山田': 3.5}

### タスク計画

In [209]:
# No(RowID), ID, タイトル, 分類, 種別, 優先度, 順番, 担当者 でCSV作成
df_tasks = pd.read_csv('./data/tasks.csv', encoding=encoding)
# タスクの基本項目は取得しておく
tasks_columns = df_tasks.columns.to_list()
df_tasks

,No,ID,タイトル,分類,種別,優先度,順番,担当者
0,1,1234,Aを調査する,残課題,調査,1,2,田中
1,2,1253,Bを調査する,残課題,調査,2,1,山田
2,3,1324,Cを研究する,要件1,研究,5,1,山田
3,4,1125,Dを実装する,要件2,実装,1,1,田中


### タスク別工程別工数（基準工数）

In [210]:
# No(タスクID), ID, タイトル, 工程名1, 工程名2, ..., 基準工数(行SUM) でCSV作成
df_tasks_detail = pd.read_csv('./data/tasks_detail.csv', encoding=encoding)
df_tasks_detail.fillna(0, inplace=True)
df_tasks_detail

,No,ID,タイトル,基本設計,詳細設計,実装,概要作成,調査,資料作成,基準工数
0,1,1234,Aを調査する,10.0,20.0,30.0,0.0,0.0,0.0,60
1,2,1253,Bを調査する,0.0,0.0,0.0,10.0,30.0,20.0,60
2,3,1324,Cを研究する,0.0,0.0,0.0,30.0,20.0,40.0,90
3,4,1125,Dを実装する,15.0,30.0,20.0,0.0,0.0,0.0,65


In [211]:
# 工程名一覧
task_detail_fields = [s for s in df_tasks_detail.columns.to_list() if not s in ['No','ID','タイトル','基準工数']]
task_detail_fields

['基本設計', '詳細設計', '実装', '概要作成', '調査', '資料作成']

### タスク別工数（係数計算後）

In [212]:
columns = task_detail_fields.copy()
columns.extend(['ID','基準工数'])
df_owner_tasks = pd.merge(df_tasks, df_tasks_detail[columns],how='left',on='ID')

# 工程別に係数をかける
for task_detail_field in task_detail_fields:
  df_owner_tasks[f'基準工数_{task_detail_field}'] = df_owner_tasks[task_detail_field]
  df_owner_tasks[task_detail_field] = df_owner_tasks.apply(lambda x: x[task_detail_field] * owner_coefficient[x.担当者], axis=1)
# 基準工数に係数をかける
df_owner_tasks["工数"] = df_owner_tasks.apply(lambda x: x.基準工数 * owner_coefficient[x.担当者], axis=1)

columns = tasks_columns.copy()
columns.extend(task_detail_fields)
columns.extend(['工数'])
df_owner_tasks[columns]

,No,ID,タイトル,分類,種別,優先度,順番,担当者,基本設計,詳細設計,実装,概要作成,調査,資料作成,工数
0,1,1234,Aを調査する,残課題,調査,1,2,田中,20.0,40.0,60.0,0.0,0.0,0.0,120.0
1,2,1253,Bを調査する,残課題,調査,2,1,山田,0.0,0.0,0.0,35.0,105.0,70.0,210.0
2,3,1324,Cを研究する,要件1,研究,5,1,山田,0.0,0.0,0.0,105.0,70.0,140.0,315.0
3,4,1125,Dを実装する,要件2,実装,1,1,田中,30.0,60.0,40.0,0.0,0.0,0.0,130.0


### スケジュール

In [213]:
# No, タイトル, 分類, 担当者, 開始日, 終了日, 単位(d/w/m), 単位あたり工数(h) でCSV作成
df_schedules = pd.read_csv('./data/schedules.csv', encoding=encoding)
df_schedules['開始日'] = pd.to_datetime(df_schedules['開始日'])
df_schedules['終了日'] = pd.to_datetime(df_schedules['終了日'])
df_schedules

,No,タイトル,分類,担当者,開始日,終了日,単位,単位あたり工数
0,1,若手MTG,本部作業,田中,2021-09-01,2021-12-31,w,5
1,2,外部研修,研修,山田,2021-09-20,2021-09-20,d,5


## シミュレート

In [214]:
format = "%Y/%m/%d"
f'PJ期間: {StartDate.strftime(format)} ～ {EndDate.strftime(format)}'

'PJ期間: 2021/09/15 ～ 2022/01/15'

In [215]:
def get_from_date(target_date, m_start):
  from_date = target_date
  if from_date < m_start: # 前月以前
    from_date = m_start
  if from_date < StartDate: # 開始日以前
    from_date = StartDate
  return from_date

def get_to_date(target_date, m_end):
  to_date = target_date
  if to_date > m_end: # 翌月以降
    to_date = m_end
  if to_date > EndDate: #終了日より後
    to_date = EndDate
  return to_date

In [216]:
def calculate_days(x):
  from_date = get_from_date(x.開始日.to_pydatetime(), m_start)
  to_date = get_to_date(x.終了日.to_pydatetime(), m_end)

  if x.単位 == 'd':
    return (to_date - from_date).days + 1
  elif x.単位 == 'w':
    return math.floor((to_date - from_date).days / 7)
  elif x.単位 == 'm':
    return 1

### 月別スケジュール別工数

In [217]:
m_start = StartDate
dfs = []
while(True):
  m_end = (m_start + relativedelta(months=1)).replace(day=1) - timedelta(days=1) #月末
  schedule_m = df_schedules.query(f"開始日 <= '{m_end}' and 終了日 >= '{m_start}'", engine='python').copy()
  
  if(len(schedule_m) != 0):
    schedule_m["月"] = m_start
    
    schedule_m["単位数"] = schedule_m.apply(lambda x: calculate_days(x), axis=1)
    schedule_m["月あたり予定工数"] = schedule_m["単位あたり工数"] * schedule_m["単位数"]
    dfs.append(schedule_m.copy())

  if EndDate <= m_end:
    break
  m_start = m_end + timedelta(days=1) # 翌月1日

df_schedule_bym = pd.concat(dfs, axis=0)
df_schedule_bym

,No,タイトル,分類,担当者,開始日,終了日,単位,単位あたり工数,月,単位数,月あたり予定工数
0,1,若手MTG,本部作業,田中,2021-09-01,2021-12-31,w,5,2021-09-15,2,10
1,2,外部研修,研修,山田,2021-09-20,2021-09-20,d,5,2021-09-15,1,5
0,1,若手MTG,本部作業,田中,2021-09-01,2021-12-31,w,5,2021-10-01,4,20
0,1,若手MTG,本部作業,田中,2021-09-01,2021-12-31,w,5,2021-11-01,4,20
0,1,若手MTG,本部作業,田中,2021-09-01,2021-12-31,w,5,2021-12-01,4,20


### 月別担当者別予定時間

In [218]:
schedule_bym_owner = df_schedule_bym.groupby(['月','担当者']).月あたり予定工数.sum().reset_index().copy()
schedule_bym_owner

,月,担当者,月あたり予定工数
0,2021-09-15,山田,5
1,2021-09-15,田中,10
2,2021-10-01,田中,20
3,2021-11-01,田中,20
4,2021-12-01,田中,20


In [219]:
hours_per_day = 6.5 #1日あたり稼働時間
days_per_week = 5
buffer_m = 0.2 #月ごとのバッファ
datetime_format = '%Y-%m-%d %H:%M:%S'
m_start = StartDate
dfs_freetime = []
while(True):
  for owner in owner_coefficient:
    m_end = (m_start + relativedelta(months=1)).replace(day=1) - timedelta(days=1) #月末
    
    from_date = get_from_date(m_start, m_start)
    to_date = get_to_date(m_end, m_end)

    weeks = math.floor((to_date - from_date).days / 7)
    
    # 月の空き時間計算
    all_time = hours_per_day * weeks * days_per_week
    buffer_time = all_time * buffer_m
    schedule_time = schedule_bym_owner.query(f"担当者=='{owner}' and 月=='{m_start}'").月あたり予定工数.sum()
    free_time = all_time - buffer_time - schedule_time
    
    dfs_freetime.append([m_start, owner, free_time, all_time, schedule_time, buffer_time])

  if EndDate <= m_end:
    break
  m_start = m_end + timedelta(days=1) # 翌月1日

df_freetime = pd.DataFrame(data=dfs_freetime, columns=["月","担当者","空き時間","月総時間","予定時間","バッファ"])
df_freetime

,月,担当者,空き時間,月総時間,予定時間,バッファ
0,2021-09-15,田中,42.0,65.0,10,13.0
1,2021-09-15,山田,47.0,65.0,5,13.0
2,2021-10-01,田中,84.0,130.0,20,26.0
3,2021-10-01,山田,104.0,130.0,0,26.0
4,2021-11-01,田中,84.0,130.0,20,26.0
5,2021-11-01,山田,104.0,130.0,0,26.0
6,2021-12-01,田中,84.0,130.0,20,26.0
7,2021-12-01,山田,104.0,130.0,0,26.0
8,2022-01-01,田中,52.0,65.0,0,13.0
9,2022-01-01,山田,52.0,65.0,0,13.0


### 月別担当者別タスク

In [220]:
hours_per_day = 6.5 #1日あたり稼働時間
days_per_week = 5
buffer_m = 0.2 #月ごとのバッファ
datetime_format = '%Y-%m-%d %H:%M:%S'
m_start = StartDate
df_unassigned = df_owner_tasks.copy()
df_unassigned["月工数"] = df_unassigned.工数
dfs_sumulate = []
while(True):
  for owner in owner_coefficient:
    m_end = (m_start + relativedelta(months=1)).replace(day=1) - timedelta(days=1) #月末
    
    # 月の空き時間計算
    free_time = df_freetime.query(f"担当者=='{owner}' and 月=='{m_start}'").空き時間.sum()
    
    # 未割当タスク
    df_unassinge_owner = df_unassigned.query(f"担当者=='{owner}'").sort_values(["優先度","順番"]).copy()
    df_unassinge_owner["累積工数"] = df_unassinge_owner.月工数.cumsum()

    # 累積工数内のタスクをアサインする
    df_assign_owner = df_unassinge_owner.query(f"累積工数<={free_time}").copy()
    df_assign_owner["月"] = m_start.replace(day=1)
    dfs_sumulate.append(df_assign_owner.copy())
    df_unassigned.drop(df_assign_owner.index, inplace=True)
    
    # 空き時間があって残タスクで一部割り当てできそうなものがあればやる
    m_summary = df_assign_owner.月工数.sum()
    df_unassinge_owner = df_unassigned.query(f"担当者=='{owner}'").sort_values(["優先度","順番"]).copy()
    if m_summary < free_time and len(df_unassinge_owner) > 0:
      devided_time = free_time - m_summary
      df_devide = df_unassinge_owner.sort_values(["優先度","順番"]).head(1)
      # 残工数を減らす
      df_unassigned.loc[df_devide.index,"月工数"] = df_devide.月工数 - devided_time
      # 空き時間分のタスクを追加する
      df_assign_owner_task = df_devide.copy()
      df_assign_owner_task["月"] = m_start
      df_assign_owner_task["月工数"] = devided_time
      dfs_sumulate.append(df_assign_owner_task.copy())

  if EndDate <= m_end:
    break
  m_start = m_end + timedelta(days=1) # 翌月1日

df_simulate = pd.concat(dfs_sumulate, axis=0)

In [221]:
columns = tasks_columns.copy() #No, ID, タイトル, 分類, 種別, 優先度, 順番, 担当者
columns.extend(['工数','月','月工数'])
df_simulate.sort_values(['月','優先度','順番'])[columns]

,No,ID,タイトル,分類,種別,優先度,順番,担当者,工数,月,月工数
3,4,1125,Dを実装する,要件2,実装,1,1,田中,130.0,2021-09-15,42.0
1,2,1253,Bを調査する,残課題,調査,2,1,山田,210.0,2021-09-15,47.0
3,4,1125,Dを実装する,要件2,実装,1,1,田中,130.0,2021-10-01,84.0
1,2,1253,Bを調査する,残課題,調査,2,1,山田,210.0,2021-10-01,104.0
3,4,1125,Dを実装する,要件2,実装,1,1,田中,130.0,2021-11-01,4.0
0,1,1234,Aを調査する,残課題,調査,1,2,田中,120.0,2021-11-01,80.0
1,2,1253,Bを調査する,残課題,調査,2,1,山田,210.0,2021-11-01,59.0
2,3,1324,Cを研究する,要件1,研究,5,1,山田,315.0,2021-11-01,45.0
0,1,1234,Aを調査する,残課題,調査,1,2,田中,120.0,2021-12-01,40.0
2,3,1324,Cを研究する,要件1,研究,5,1,山田,315.0,2021-12-01,104.0


## シミュレート結果まとめ

### 未割当タスクが残っていないことの確認

In [222]:
columns = tasks_columns.copy() #No, ID, タイトル, 分類, 種別, 優先度, 順番, 担当者
columns.extend(['工数','月工数'])
df_unassinge_owner[columns].query('月工数!=0')

,No,ID,タイトル,分類,種別,優先度,順番,担当者,工数,月工数
2,3,1324,Cを研究する,要件1,研究,5,1,山田,315.0,166.0


### タスク別割り当て期間

In [223]:
df_term = df_simulate.groupby('No').月.agg({'min','max'}).reset_index()
df_term.rename(columns={'min': '開始月', 'max': '終了月'}, inplace=True)
df_term["期間"] = df_term.apply(lambda x: f'{x.開始月.month}-{x.終了月.month}', axis=1)

columns = tasks_columns.copy() #No, ID, タイトル, 分類, 種別, 優先度, 順番, 担当者
columns.extend(['工数','期間'])
df_task_term = pd.merge(df_owner_tasks, df_term, on='No', how='left')[columns]

In [224]:
df_task_term_detail = pd.merge(df_task_term, df_tasks_detail.drop(['ID','タイトル'], axis=1), on='No', how='left')
columns = ['期間']
columns.extend(tasks_columns) #No, ID, タイトル, 分類, 種別, 優先度, 順番, 担当者
columns.extend(task_detail_fields)
columns.append('基準工数')
df_task_term_detail_out = df_task_term_detail.sort_values(['分類','優先度','順番'])[columns]
df_task_term_detail_out

,期間,No,ID,タイトル,分類,種別,優先度,順番,担当者,基本設計,詳細設計,実装,概要作成,調査,資料作成,基準工数
0,11-12,1,1234,Aを調査する,残課題,調査,1,2,田中,10.0,20.0,30.0,0.0,0.0,0.0,60
1,9-11,2,1253,Bを調査する,残課題,調査,2,1,山田,0.0,0.0,0.0,10.0,30.0,20.0,60
2,11-1,3,1324,Cを研究する,要件1,研究,5,1,山田,0.0,0.0,0.0,30.0,20.0,40.0,90
3,9-11,4,1125,Dを実装する,要件2,実装,1,1,田中,15.0,30.0,20.0,0.0,0.0,0.0,65


### スケジュール別総工数

In [225]:
df_schedule_out = pd.merge(df_schedules,df_schedule_bym.groupby('No').月あたり予定工数.sum().reset_index(),how='left',on='No').rename(columns={'月あたり予定工数':'総工数'})
df_schedule_out

,No,タイトル,分類,担当者,開始日,終了日,単位,単位あたり工数,総工数
0,1,若手MTG,本部作業,田中,2021-09-01,2021-12-31,w,5,70
1,2,外部研修,研修,山田,2021-09-20,2021-09-20,d,5,5


### 月別担当者別負荷

In [226]:
columns = ['月','担当者','月総時間','予定時間','月工数','バッファ']
df_simulate_bym = pd.merge(df_freetime,df_simulate.groupby(['月','担当者']).月工数.sum().reset_index(),how='left',on=['月','担当者'])[columns].fillna(0)
df_simulate_bym['バッファ'] = df_simulate_bym.月総時間 - df_simulate_bym.予定時間 - df_simulate_bym.月工数
df_simulate_bym['バッファ割合'] = df_simulate_bym.バッファ / df_simulate_bym.月総時間
df_simulate_bym

,月,担当者,月総時間,予定時間,月工数,バッファ,バッファ割合
0,2021-09-15,田中,65.0,10,42.0,13.0,0.200000
1,2021-09-15,山田,65.0,5,47.0,13.0,0.200000
2,2021-10-01,田中,130.0,20,84.0,26.0,0.200000
3,2021-10-01,山田,130.0,0,104.0,26.0,0.200000
4,2021-11-01,田中,130.0,20,84.0,26.0,0.200000
5,2021-11-01,山田,130.0,0,104.0,26.0,0.200000
6,2021-12-01,田中,130.0,20,40.0,70.0,0.538462
7,2021-12-01,山田,130.0,0,104.0,26.0,0.200000
8,2022-01-01,田中,65.0,0,0.0,65.0,1.000000
9,2022-01-01,山田,65.0,0,52.0,13.0,0.200000


### 担当者別総負荷

In [227]:
df_owner_out = df_simulate_bym.groupby('担当者').sum().reset_index().drop('バッファ割合', axis=1)
df_owner_out

,担当者,月総時間,予定時間,月工数,バッファ
0,山田,520.0,5,411.0,104.0
1,田中,520.0,70,250.0,200.0


In [228]:
df_owner_tasks_out_list = []
for task_detail_field in task_detail_fields:
  columns = tasks_columns.copy()
  columns.append(task_detail_field)
  df_add = df_owner_tasks[columns].rename(columns={task_detail_field: '工数'}).query('工数!=0').copy()
  df_add['タスク名'] = task_detail_field
  df_add['実績'] = 0
  df_add['残工数'] = 0
  df_owner_tasks_out_list.append(df_add)

columns = tasks_columns.copy()
columns.extend(['タスク名','工数','実績','残工数'])
df_task_time_out = pd.concat(df_owner_tasks_out_list, axis=0)[columns].sort_values(['優先度','順番','ID'])
df_task_time_out

,No,ID,タイトル,分類,種別,優先度,順番,担当者,タスク名,工数,実績,残工数
3,4,1125,Dを実装する,要件2,実装,1,1,田中,基本設計,30.0,0,0
3,4,1125,Dを実装する,要件2,実装,1,1,田中,詳細設計,60.0,0,0
3,4,1125,Dを実装する,要件2,実装,1,1,田中,実装,40.0,0,0
0,1,1234,Aを調査する,残課題,調査,1,2,田中,基本設計,20.0,0,0
0,1,1234,Aを調査する,残課題,調査,1,2,田中,詳細設計,40.0,0,0
0,1,1234,Aを調査する,残課題,調査,1,2,田中,実装,60.0,0,0
1,2,1253,Bを調査する,残課題,調査,2,1,山田,概要作成,35.0,0,0
1,2,1253,Bを調査する,残課題,調査,2,1,山田,調査,105.0,0,0
1,2,1253,Bを調査する,残課題,調査,2,1,山田,資料作成,70.0,0,0
2,3,1324,Cを研究する,要件1,研究,5,1,山田,概要作成,105.0,0,0


In [231]:
# 出力
df_task_term_detail_out.to_csv('./out/silumate_tasks.csv', index=False)
df_schedule_out.to_csv('./out/schedule.csv', index=False)
df_schedule_bym.to_csv('./out/suchedule_bym.csv', index=False)
df_task_time_out.to_csv('./out/task_time.csv', index=False)

In [233]:
columns = tasks_columns.copy() #No, ID, タイトル, 分類, 種別, 優先度, 順番, 担当者
columns.extend(['工数','月','月工数'])
df_simulate.sort_values(['月','優先度','順番'])[columns].to_csv('./out/simulate_tasks_bym.csv', index=False)